In [1]:
# -*- coding: UTF-8 -*-

import numpy
import jieba
import jieba.analyse
import pandas as pd
import codecs
import csv

In [2]:
data = pd.read_csv('data/消费content16.csv', encoding='gb18030')


In [3]:
def load_stopword():
    '''
    加载停用词表
    :return: 返回停用词的列表
    '''
    stopword_path = 'data/scu_stopwords.txt'
    f_stop = open(stopword_path,'r',encoding='UTF-8')
    sw = [line.strip() for line in f_stop]
    f_stop.close()
    stopword_extend = ['\n','湖北','武汉','病毒','中国','兰州','甘肃','18','我国','北京','10','微博']
    sw.extend(stopword_extend)
    return sw

In [4]:
def jieba_text(data):
    stopwords = load_stopword()
    df = data
    text_seg_list = []
    for index,row in df.iterrows():
        fileId = row.index
        fileContent = row[0]
        segs = jieba.analyse.textrank(fileContent,topK=20,withWeight=False,allowPOS=('ns','n','nt'))
        segments_list = []
        for seg in segs:
            if seg not in stopwords and len(seg) > 1:
                segments_list.append(seg)
        text_seg_list.append(segments_list)
    return text_seg_list

In [5]:
jieba_text(data)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\李大帅\AppData\Local\Temp\jieba.cache
Loading model cost 0.803 seconds.
Prefix dict has been built successfully.


[['协会', '硬核'],
 ['尾号', '靓号', '河北', '人员', '新闻', '违约金', '客服', '营业厅', '号码', '协议', '合约'],
 ['评论', '超前消费', '鹰式', '负债', '力量', '穷人', '斜杠', '信用', '商业'],
 ['压力', '财经', '男生', '高压', '人群', '女生', '比例', '学历', '来源'],
 ['压力', '老板', '男生', '高压', '人群', '比例', '学历', '来源', '女生'],
 ['核心', '导向', '案例', '金句', '主题'],
 ['压力', '高压', '人群', '老板', '学历', '来源'],
 ['物品', '心理', '效应', '美国哈佛大学', '过度', '经济学家', '专指', '财经', '相适应', '现象'],
 ['洞察', '雪树', '利率', '本钱'],
 ['税率',
  '进口关税',
  '商品',
  '药品',
  '食用',
  '配方',
  '产业',
  '电子',
  '最惠国',
  '日用',
  '高新技术',
  '生物',
  '机动车',
  '变速箱',
  '液力',
  '压力',
  '冻猪肉',
  '用药',
  '新药'],
 ['日本', '绿洲', '东京'],
 ['压力', '财经', '男生', '高压', '人群', '女生', '比例', '学历', '来源'],
 ['运气', '实力', '老板'],
 ['摄像头', '解决方案', '竞争力', '事物', '核心', '指纹', '屏幕', '海量', '优势', '双屏', '无法'],
 ['词汇', '节点', '网络', '体感', '边界', '本年度', '优势', '技术人员', '洞察', '核心'],
 [],
 ['年轻人',
  '差异',
  '父辈',
  '花钱',
  '方法',
  '父母',
  '日子',
  '时代',
  '消费观念',
  '心思',
  '上多花',
  '观念',
  '细节',
  '爷爷奶奶',
  '规则',
  '朋友圈'],
 ['老板', '运气', '实力'],
 ['手机', '高铁

In [6]:
def LDA(text_seg_list,topic_num):
    # 建立字典
    dictionary = corpora.Dictionary(text_seg_list)
    V = len(dictionary)

    # 转换文本数据为索引，并计数
    corpus = [dictionary.doc2bow(text) for text in text_seg_list]

    # 计算tf-idf值
    corpus_tfidf = models.TfidfModel(corpus)[corpus]

    # 训练模型
    lda = models.LdaModel(corpus_tfidf, num_topics=topic_num, id2word=dictionary)
    #alpha=0.01, eta=0.01, minimum_probability=0.001,update_every=1, chunksize=100, passes=1
    Perplexity = lda.log_perplexity(corpus_tfidf)

    num_show_term = 10  # 每个主题显示几个词
    #print('结果：每个主题的词分布：')
    lda_topic = {'1':[],'2':[],'3':[],'4':[],'5':[],'6':[],'7':[],'8':[],'9':[],'10':[]}
    lda_topic_prob = []
    for topic_id in range(topic_num):
        #print('主题#%d：\t' % topic_id)
        term_distribute_all = lda.get_topic_terms(topicid=topic_id)
        term_distribute = term_distribute_all[:num_show_term]
        term_distribute = np.array(term_distribute)
        term_id = term_distribute[:, 0].astype(np.int)
        #print('词：\t', )
        i = 0
        for t in term_id:
            i += 1
            #print(dictionary.id2token[t], )
            lda_topic[str(i)].append(dictionary.id2token[t])
        #print('\n概率：\t', term_distribute[:, 1])
        lda_topic_prob.append(term_distribute[:, 1])
    print(lda.print_topics(5))
    print('123')
    
    #lda可视化
    vis_data = pyLDAvis.gensim.prepare(lda,corpus_tfidf,dictionary)
    pyLDAvis.show(vis_data,open_browser = False)
    return lda_topic,lda_topic_prob,Perplexity

In [7]:
import numpy as np
import gensim
from gensim import corpora, models, similarities
from gensim.models.doc2vec import Doc2Vec, LabeledSentence
from sklearn import metrics
from sklearn.cluster import KMeans
from pprint import pprint
import time
import jieba
import jieba.analyse
import pandas as pd
from sklearn.manifold import TSNE
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
LDA(jieba_text(data), 10)

[(5, '0.037*"机制" + 0.025*"指南" + 0.017*"风险" + 0.011*"下单" + 0.010*"消费观" + 0.008*"食品" + 0.007*"课堂" + 0.007*"普法" + 0.006*"夜市" + 0.005*"年轻人"'), (9, '0.030*"水龙头" + 0.027*"指南" + 0.022*"人群" + 0.022*"生肉" + 0.020*"食品" + 0.013*"直播间" + 0.012*"现金" + 0.010*"常态" + 0.007*"水果" + 0.006*"口味"'), (7, '0.051*"国务院" + 0.012*"涨幅" + 0.011*"代言" + 0.011*"大盘" + 0.008*"科技股" + 0.008*"资金" + 0.007*"儿童" + 0.007*"钢铁" + 0.007*"情绪" + 0.006*"医药"'), (8, '0.010*"头条" + 0.010*"网购" + 0.009*"文章" + 0.008*"消费税" + 0.007*"山东" + 0.007*"增值税" + 0.007*"母亲" + 0.006*"理想" + 0.006*"办法" + 0.005*"委员"'), (2, '0.020*"肺炎" + 0.011*"新冠" + 0.008*"饮料" + 0.008*"地区" + 0.007*"直言" + 0.006*"热情" + 0.006*"回家" + 0.006*"经营者" + 0.006*"代言人" + 0.006*"商务"')]
123


D:\ANACONDA\envs\statistic\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))



Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]
